In [1]:
import pandas as pd
import os
os.chdir("..")
os.chdir("C:/Users/Amin/Downloads")
cwd = os.getcwd()

print("Current Working Directory:", cwd)

Current Working Directory: C:\Users\Amin\Downloads


In [2]:
start_index = 0
end_index = 1000

In [3]:
dtype_spec = {
    'gvkey': 'string',
}
df = pd.read_csv(f"aggregated_dataset{start_index}_{end_index}.csv", dtype=dtype_spec)
# these columns are redundant or exact copies. For each, There is another column with a different name with the exact same information.
df = df.drop(['date', 'permno', 'permco', 'iid', 'id', 'excntry', 'obs_main',
              'exch_main', 'primary_sec', 'common', 'ret_lag_dif', 'curcd',
              'fx', 'comp_tpci', 'crsp_shrcd','source_crsp','me_company',
              'ret_local','prc_local','me','ret',
              ], axis=1)
print(len(df))

C:\Users\Amin\AppData\Local\Temp\ipykernel_17696\3832074588.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"aggregated_dataset{start_index}_{end_index}.csv", dtype=dtype_spec)


221699


In [17]:
# I need to calculate 36month betas. So, I can delete stocks that have less than 3 years of data.
group_sizes = df.groupby('gvkey').transform('size')
mask = group_sizes >= 37
filtered_df = df[mask]
print(len(filtered_df))
filtered_df.head(5)

219360


,gvkey,eom,size_grp,ret_exc_lead1m,ret_exc,prc_high,prc_low,bidask,gics,naics,...,dltnetis_mev,dstnetis_mev,dbnetis_mev,netis_mev,fincf_mev,ivol_capm_60m,beta_21d,beta_252d,rvol_252d,rvolhl_21d
0,001000,1970-09-30,micro,0.058876,NaN,NaN,NaN,0.0,NaN,NaN,...,0.000713,0.213338,0.214051,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001000,1970-10-31,micro,-0.284489,0.058876,NaN,NaN,0.0,NaN,NaN,...,0.000679,0.203243,0.203922,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001000,1970-11-30,micro,0.107325,NaN,10.875,8.750,0.0,NaN,NaN,...,0.000765,0.229035,0.229800,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001000,1970-12-31,micro,-0.103777,0.107325,10.125,7.625,0.0,NaN,NaN,...,0.000706,0.211159,0.211865,NaN,NaN,NaN,0.638698,NaN,NaN,0.034683
4,001000,1971-01-31,micro,0.232767,-0.103777,10.000,8.750,0.0,NaN,NaN,...,0.000765,0.229035,0.229800,NaN,NaN,NaN,4.439515,NaN,NaN,0.024832


In [11]:
mom_df = pd.read_csv("data/F-F_Momentum_Factor.CSV")
mom_df.head(5)

,date,Mom
0,192701,0.36
1,192702,-2.14
2,192703,3.61
3,192704,4.30
4,192705,3.00


In [19]:
ff5_df = pd.read_csv("data/F-F_Research_Data_5_Factors_2x3.CSV")
ff5_df['date'] = pd.to_datetime(ff5_df['date'] + '01', format='%Y%m%d') + pd.offsets.MonthEnd(0)
ff5_df.head(6)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U2')) -> None

In [21]:
factor_df = pd.merge(ff5_df,mom_df,how="left").drop(['RF'], axis=1)

factor_df['date'] = factor_df['date'].astype(str)
# Ensure all date strings have the correct length (6 characters)
factor_df['date'] = factor_df['date'].str.zfill(6)  # This ensures leading zeros are added if necessary
# Convert to datetime format (yyyy-mm-dd) with the last day of the month
factor_df['date'] = pd.to_datetime(factor_df['date'] + '01', format='%Y%m%d') + pd.offsets.MonthEnd(0)
factor_df.head(12)


,date,Mkt-RF,SMB,HML,RMW,CMA,Mom
0,1963-07-31,-0.39,-0.41,-0.97,0.68,-1.18,0.90
1,1963-08-31,5.07,-0.80,1.80,0.36,-0.35,1.01
2,1963-09-30,-1.57,-0.52,0.13,-0.71,0.29,0.19
3,1963-10-31,2.53,-1.39,-0.10,2.80,-2.01,3.12
4,1963-11-30,-0.85,-0.88,1.75,-0.51,2.24,-0.74
5,1963-12-31,1.83,-2.10,-0.02,0.03,-0.07,1.75
6,1964-01-31,2.24,0.13,1.48,0.17,1.47,0.86
7,1964-02-29,1.54,0.28,2.81,-0.05,0.91,0.26
8,1964-03-31,1.41,1.23,3.40,-2.21,3.22,0.75
9,1964-04-30,0.10,-1.52,-0.67,-1.27,-1.08,-0.58
